# Movie Collaborative Recommender

### Libraries
- https://pandas.pydata.org/
- http://www.numpy.org/
- https://docs.python.org/3/library/pickle.html

### Source
- https://www.linkedin.com/learning/machine-learning-ai-foundations-recommendations

In [1]:
# Configuration

# user_id to get recommendations (Between 1 and 100):")
user_id_to_search = 1

In [2]:
# Libraries

import os
import webbrowser
import numpy as np
import pandas as pd
import pickle
import matrix_factorization_utilities

In [3]:
# Load data

# Load user ratings
raw_dataset_df = pd.read_csv('data/movie_ratings_data_set.csv')

# Load movie titles
movies_df = pd.read_csv('data/movies.csv', index_col='movie_id')

# Convert the running list of user ratings into a matrix
ratings_df = pd.pivot_table(raw_dataset_df, index='user_id',
                            columns='movie_id',
                            aggfunc=np.max)

In [4]:
# Build model

# Apply matrix factorization to find the latent features
U, M = matrix_factorization_utilities.low_rank_matrix_factorization(ratings_df.as_matrix(),
                                                                    num_features=15,
                                                                    regularization_amount=0.1)

# Find all predicted ratings by multiplying U and M matrices
predicted_ratings = np.matmul(U, M)

# Save all the ratings to a csv file
predicted_ratings_df = pd.DataFrame(index=ratings_df.index,
                                    columns=ratings_df.columns,
                                    data=predicted_ratings)
predicted_ratings_df.to_csv("output/predicted_ratings.csv")

# Save features and predicted ratings to files for later use
pickle.dump(U, open("models/user_features.dat", "wb"))
pickle.dump(M, open("models/product_features.dat", "wb"))
pickle.dump(predicted_ratings, open("models/predicted_ratings.dat", "wb" ))


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


         Current function value: 32.504364
         Iterations: 3000
         Function evaluations: 4477
         Gradient evaluations: 4477


In [5]:
# Predict

# Load prediction rules from data files
U = pickle.load(open("models/user_features.dat", "rb"))
M = pickle.load(open("models/product_features.dat", "rb"))
predicted_ratings = pickle.load(open("models/predicted_ratings.dat", "rb"))

# Load movie titles
movies_df = pd.read_csv('data/movies.csv', index_col='movie_id')

print("Movies previously reviewed by user_id {}:".format(user_id_to_search))

reviewed_movies_df = raw_dataset_df[raw_dataset_df['user_id'] == user_id_to_search]
reviewed_movies_df = reviewed_movies_df.join(movies_df, on='movie_id')

print(reviewed_movies_df[['title', 'genre', 'value']])

print("Movies we will recommend:")

user_ratings = predicted_ratings[user_id_to_search - 1]
movies_df['rating'] = user_ratings

already_reviewed = reviewed_movies_df['movie_id']
recommended_df = movies_df[movies_df.index.isin(already_reviewed) == False]
recommended_df = recommended_df.sort_values(by=['rating'], ascending=False)

print(recommended_df[['title', 'genre', 'rating']].head(5))

Movies previously reviewed by user_id 1:
               title                  genre  value
0      The Sheriff 4   crime drama, western      4
1  Mafia Underground  crime drama, thriller      4
2        Biker Gangs    crime drama, action      4
3      The Sheriff 1   crime drama, western      4
4     The Spy Family              spy drama      4
5      The Sheriff 3   crime drama, western      5
Movies we will recommend:
                              title                  genre    rating
movie_id                                                            
34            The Serious Detective        detective drama  4.570281
3                     The Sheriff 2   crime drama, western  4.155678
6                 Attack on Earth 1         sci-fi, action  4.142751
19        Fake News about Fake News         satire, comedy  4.119082
8          Sci-Fi Murder Detectives  supernatural, mystery  4.066306


In [6]:
# Mesure accuracy with training, testing datasets

# Load user ratings
raw_training_dataset_df = pd.read_csv('data/movie_ratings_data_set_training.csv')
raw_testing_dataset_df = pd.read_csv('data/movie_ratings_data_set_testing.csv')

# Convert the running list of user ratings into a matrix
ratings_training_df = pd.pivot_table(raw_training_dataset_df, index='user_id', columns='movie_id', aggfunc=np.max)
ratings_testing_df = pd.pivot_table(raw_testing_dataset_df, index='user_id', columns='movie_id', aggfunc=np.max)

# Apply matrix factorization to find the latent features
U, M = matrix_factorization_utilities.low_rank_matrix_factorization(ratings_training_df.as_matrix(),
                                                                    num_features=11,
                                                                    regularization_amount=1.1)

# Find all predicted ratings by multiplying U and M
predicted_ratings = np.matmul(U, M)

# Measure RMSE
rmse_training = matrix_factorization_utilities.RMSE(ratings_training_df.as_matrix(),
                                                    predicted_ratings)
rmse_testing = matrix_factorization_utilities.RMSE(ratings_testing_df.as_matrix(),
                                                   predicted_ratings)

print("Training RMSE: {}".format(rmse_training))
print("Testing RMSE: {}".format(rmse_testing))


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


Optimization terminated successfully.
         Current function value: 315.538580
         Iterations: 875
         Function evaluations: 1306
         Gradient evaluations: 1306
Training RMSE: 0.24952560869598406
Testing RMSE: 1.2096509829672488


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [7]:
# Create a web page view of the data for easy viewing
html = raw_dataset_df[0:100].to_html()

# Save the html to a temporary file
with open("output/data.html", "w") as f:
    f.write(html)

# Open the web page in our web browser
full_filename = os.path.abspath("output/data.html")
webbrowser.open("file://{}".format(full_filename))

True

In [8]:
# Create a web page view of the movie list for easy viewing
html = movies_df.to_html()

# Save the html to a temporary file
with open("output/movie_list.html", "w") as f:
    f.write(html)

# Open the web page in our web browser
full_filename = os.path.abspath("output/movie_list.html")
webbrowser.open("file://{}".format(full_filename))

True

In [9]:
# Create a web page view of the review matrix for easy viewing
html = ratings_df.to_html(na_rep="")

# Save the html to a temporary file
with open("output/review_matrix.html", "w") as f:
    f.write(html)

# Open the web page in our web browser
full_filename = os.path.abspath("output/review_matrix.html")
webbrowser.open("file://{}".format(full_filename))

True